## Load packages

In [1]:
# Load packages
import pandas as pd
import numpy as np

## Load datasets

In [2]:
# Load the main dataset
df = pd.read_pickle("data/pkl/extracted-quotes-2018.pkl")

In [229]:
# Load the dataset with additional info abbout politicians 
df_politicians = pd.io.json.read_json("data/resources/congress_biolist.json")

## Get a nice table with politician's feautures

In [261]:
# Add a new column with First and Family name 
df_politicians["fullName"] = df_politicians["givenName"] + ' ' + df_politicians["familyName"]
df_politicians["speaker"] = df_politicians["givenName"] + ' ' + df_politicians["familyName"]

In [231]:
# Work on the column "congresses"
df_politicians['congresses'] = df_politicians['congresses'].astype(str)
df_politicians['congresses'] = df_politicians['congresses'].str.replace('[', '', regex=True)
df_politicians['congresses'] = df_politicians['congresses'].str.replace(']', '', regex=True)

In [232]:
congress_df = df_politicians['congresses'].str.split(',',4, expand=True)
congress_df = congress_df.rename(columns={0: 'Position', 1: 'Congress Number', 2:'State Name',3:'Parties'})
congress_df = congress_df.drop(4, axis=1)

In [233]:
# As I never managed to split the column differently I did it with strings 
congress_df['Position'] = congress_df['Position'].str.replace('{', '', regex=True)
congress_df['Position'] = congress_df['Position'].str.replace('position', '', regex=True)
congress_df['Position'] = congress_df['Position'].str.replace(':', '', regex=True)
congress_df['Position'] = congress_df['Position'].str.replace("'", "")

congress_df['Congress Number'] = congress_df['Congress Number'].str.replace('congressNumber', '', regex=True)
congress_df['Congress Number'] = congress_df['Congress Number'].str.replace(':', '', regex=True)
congress_df['Congress Number'] = congress_df['Congress Number'].str.replace("'", "")

congress_df['State Name'] = congress_df['State Name'].str.replace('stateName', '', regex=True)
congress_df['State Name'] = congress_df['State Name'].str.replace(':', '', regex=True)
congress_df['State Name'] = congress_df['State Name'].str.replace("'", "")

congress_df['Parties'] = congress_df['Parties'].str.replace('parties', '', regex=True)
congress_df['Parties'] = congress_df['Parties'].str.replace('}', '', regex=True)
congress_df['Parties'] = congress_df['Parties'].str.replace(':', '', regex=True)
congress_df['Parties'] = congress_df['Parties'].str.replace("'", "")


Concatenate the table

In [234]:
df_politicians = pd.concat([df_politicians, congress_df], axis=1)
df_politicians.head(3)

,id,givenName,familyName,unaccentedGivenName,unaccentedFamilyName,birthYear,deathYear,congresses,middleName,unaccentedMiddleName,nickName,honorificPrefix,honorificSuffix,fullName,Position,Congress Number,State Name,Parties
0,A000014,Neil,Abercrombie,Neil,Abercrombie,1938,NaN,"{'position': 'Representative', 'congressNumber...",NaN,NaN,NaN,NaN,NaN,Neil Abercrombie,Representative,99,HI,Democrat
1,A000374,Ralph,Abraham,Ralph,Abraham,1954,NaN,"{'position': 'Representative', 'congressNumber...",NaN,NaN,NaN,NaN,NaN,Ralph Abraham,Representative,115,LA,Republican
2,A000022,Gary,Ackerman,Gary,Ackerman,1942,NaN,"{'position': 'Representative', 'congressNumber...",Leonard,Leonard,NaN,NaN,NaN,Gary Ackerman,Representative,98,NY,Democrat


## Now we want to merge the table with speakers with the table ith info about them

Check hther the politicians have duplicates. We have 4 cases where 2 politicians have the same name. I am deleting those names form the main dataset as we don't know about which one we will talk.

In [257]:
len(df_politicians)

1147

In [259]:
df_politicians['fullName'].value_counts()

Neil Abercrombie      1
Markwayne Mullin      1
Stephanie Murphy      1
Scott Murphy          1
Christopher Murphy    1
                     ..
Jesús García          1
Cory Gardner          1
Thomas Garrett        1
Scott Garrett         1
Ryan Zinke            1
Name: fullName, Length: 1147, dtype: int64

In [245]:
df_politicians = df_politicians[df_politicians["fullName"].str.contains("Timothy Johnson")==False]
df_politicians = df_politicians[df_politicians["fullName"].str.contains("Donald Payne")==False]
df_politicians = df_politicians[df_politicians["fullName"].str.contains("Duncan Hunter")==False]
df_politicians = df_politicians[df_politicians["fullName"].str.contains("Patrick Murphy")==False]

In [260]:
len(df_politicians)

1147

I should have the name and family name in title (capital letter for every word) in order to have a better fit.

In [265]:
df_politicians["speaker"] = df_politicians["speaker"].str.title()
df["speaker"] = df["speaker"].str.title()

In [296]:
data = pd.merge(df, df_politicians, on='speaker', how='outer') # we merged the dataset

In [312]:
## We should drop quotes that didn't math the politician 
data = data[data['Parties'].notna()]

In [314]:
# Subsets by parties 
subset_democrats = data[data['Parties'].str.contains("Democrat")]
subset_republican = data[data['Parties'].str.contains("Republican")]

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,subset,...,middleName,unaccentedMiddleName,nickName,honorificPrefix,honorificSuffix,fullName,Position,Congress Number,State Name,Parties


0